In [8]:
from numpy import random
import numpy as np
from scipy import optimize
template_sd = 1
deformation_sd = 1
# alphas = 1
# betas = [1, 1]
ag = 1
n = 1
kp = 2
kg = 2

def kernel_function(a,center,sd) -> int:
        return np.exp(-(np.linalg.norm(a-center)**2)
               /(2*sd **2))

sigmaP = np.array([[0,0],
                 [0,0]])
sigmaPinv = np.array([[0,0],
                 [0,0]])

sigmaG = np.array([[0, 0],
                   [0, 0]])
sigmaGinv = np.array([[0, 0],
                   [0, 0]])


p_centers = np.array([0,1])
g_centers = np.array([0,1])


def cal_deformation(x,b):
    sumer = 0
    for index,center in enumerate(g_centers):
        sumer += b[index] * kernel_function(x,center,deformation_sd)
    return sumer

for i in range(kp):
    for j in range(kp):
        sigmaPinv[i,j] = kernel_function(p_centers[i],
                                      p_centers[j],
                                      template_sd)

for i in range(kg):
    for j in range(kg):
        sigmaGinv[i,j] = kernel_function(g_centers[i],
                                      g_centers[j],
                                      deformation_sd)


sigmaP = np.linalg.inv(sigmaPinv)
sigmaG = np.linalg.inv(sigmaGinv)


def calculate_kBp(b):
        tmp_kbp = np.empty((5, kp))
        for i in range(5):
            for j in range(kp):
                the_def = cal_deformation(i,b)
                tmp_kbp[i,j] = kernel_function(i - the_def,
                                                p_centers[j],
                                                template_sd)
        return tmp_kbp

In [6]:
class MyImages:
    def __init__(self,a):
        self.location = a.ded
        # then load the images
        self.img = []

    def get_images(self):
        pass


class TemplateKernel:
    def __init__(self,center):
        self.sd = template_sd
        self.center = center

    def kernel_function(self,a) -> int:
        return np.exp(-(np.linalg.norm(a-self.center))**2
               /(2* self.sd **2))

class DeformationKernel:
    def __init__(self,center):
        self.sd = 1
        self.center = center

    def kernel_function(self,a) -> int:
        return np.exp(-(np.linalg.norm(a-self.center))**2
               /(2* self.sd **2))


# Intro
I have no idea what is happening

In [5]:
class Estimator:

    def __init__(self):
        self.alphas = np.array([0,0])
        self.betas = np.array([0,0])
        self.sd = 1
        self.kBp = \
            calculate_kBp(self.betas)
        self.Gamma = sigmaG
        self.images = np.array([[0,1,1,1,0],
                                [1,0,1,1,0]])
        self.predictions = np.array([0,0,0,0,0])

    def to_minimize(self,b):
        1/2 * np.matmul(np.matmul(b.T,np.linalg.inv(self.Gamma)),
                        b) + \
        1/(2*self.sd) * np.linalg.norm(self.images - self.predictions)**2


    def best_betas(self):
        out = optimize.minimize(self.to_minimize,self.betas).x
        self.betas = out
        return out

    def bbtl(self):
        betas = self.best_betas()
        np.matmul(betas,betas.T)

    def update_Gigma(self):
        self.Gamma = 1/( n+ ag) * (n* self.bbtl() + ag * sigmaG)

    def update_prediction(self):
        self.predictions = np.matmul(self.kBp,self.alphas)

    def update_alpha(self):
        pass

    def yTy(self):
        np.linalg.norm(self.images)**2

    def ky_kk(self):
        b = self.best_betas()
        kBp = calculate_kBp(b)
        ky = np.matmul(kBp.T,self.images)
        kk = np.matmul(kBp.T,kBp)
        return ky,kk

    # def new_alpha(self):



In [ ]:
k = 10

def tmp_1d(x) -> int:
    if abs(x) <= 10:
        return 1
    else:
        return 0

def noisy_tmp_1d(x) -> int:
    return tmp_1d(x + random.normal(0,1))


#
# def noisy_tmp(x) -> int:
#     return add_noise(tmp_1d)(x)
